In [ ]:
################## IMPORTS ##################

import requests
import re
import json
import datetime
import time
from getpass import getpass

################## FUNCTIONS ##################

# using authentication [user,pass] to get a token from the historian
def getUserToken(auth,url):
    method = 'getUserToken'
    payload = "{\r\n  \"username\":\"%s\",\r\n  \"password\":\"%s\"\r\n}"%(auth[0],auth[1])
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        return userToken
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

# using the getSenderUserToken and a post to the getSessionToken endpoint to return both tokens
def getSenderTokens(url):
    method = 'getSessionToken'
    # requesting user and pass
    print('\nEnter your username: ')
    user = input()
    print('\nEnter your password: ')
    pswd = getpass()
    auth = [user,pswd]
    userToken = getUserToken(auth,url)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"historians\":[\"historian.domain.com\"],\r\n  \"clientId\":\"pythonWriteAPI\",\r\n  \"settings\":{\r\n  \t\"clientTimeout\":300000,\r\n  \t\"fileSize\":8,\r\n  \t\"autoCreateDatasets\": true,\r\n  \t\"autoWriteNoData\": false\r\n  }\r\n}"%userToken
    headers = {
      'Content-Type': 'application/json',
      'Content-Type': 'text/plain'
    }
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    sessToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        print('I was able to get the User and Session tokens for the Write API.')
        return (userToken,sessToken)
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))
        return ('error','error')

# using authentication [user,pass] to get a token from the historian
def revokeSessionToken(utoken,stoken):
    method = 'revokeSessionToken'
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"sessionToken\":\"%s\"\r\n}"%(utoken,stoken)
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        print('\nThe Write API tokens have been revoked.')
        return 0,0
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

# function for making the payload for a storeData post of a single TVQ
def storeDataMakePayload(tag,timestamp,value,utoken,stoken):
    payload = "%s\": [\r\n  \t\t[\r\n  \t\t\"%s\",\r\n  \t\t%s\r\n  \t\t]\r\n  \t]\r\n  }\r\n}"%(tag,timestamp,value)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"sessionToken\":\"%s\",\r\n  \"tvqs\": {\r\n  \t\""%(utoken,stoken) + payload
    return payload
################## EXAMPLE FOR USING FUNCTIONS ##################

def tagValue(tagName, msg, tagPath, url, timestamp, utoken, stoken):
    method = 'storeData'
    headers = {
       'Content-Type': 'application/json',
       'Content-Type': 'text/plain'
    }
    value = msg[tagName]
    if isinstance(value, str):
        value = '"' + value +'"'
    elif isinstance(value, bool):
        value = str(value).lower()
    elif value is None:
        value = "\"no value\""
        
    tag = tagPath + tagName
    payload = storeDataMakePayload(tag,timestamp,value,utoken,stoken)
    #print(payload)
    response = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    parsed = json.loads(response)
    if parsed.get('statusCode') == 'Good':
        print(response)
    else:
        print(payload)
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

def storeDataOlympus(url):
    date_format = '%Y-%m-%dT%H:%M:%S'
    dataset = 'sandbox.'
    data = []
    utoken,stoken = getSenderTokens(url)
    print('\nEnter the full path to the payload file, including file name extension (i.e. include .json or .txt): ')
    filepath = input()
    name = '.'.join(filepath.split('\\')[-1].split('.')[:-1])
    with open(filepath) as o:
        wer = o.read()
        wer = '{' + '{'.join(wer.split('{')[1:])
        msg = json.loads(wer)
    try:
        #grab timestamp
        timestamp = msg['TimeStamp']

        #create tag path for header into
        area = 'BWI.AFF.Olympus.'
        equip = msg['Source']+'.'
        cfxClass = msg['MessageName']+'.'
        recipeName = msg['MessageBody']['RecipeName']+'.'
        recipeRev = msg['MessageBody']['RecipeRevision']+'.'
        tagPath = dataset+area+equip+cfxClass+recipeName+recipeRev

        #create header tags GOOD
        tagValue('Version', msg, tagPath, url, timestamp, utoken, stoken)
        tagValue('TransactionId', msg['MessageBody'], tagPath, url, timestamp, utoken, stoken)
        tagValue('InspectionMethod', msg['MessageBody'], tagPath, url, timestamp, utoken, stoken)

        #iterating through samplingInfo dict
        for key in msg['MessageBody']['SamplingInformation']:
            tagValue(key, msg['MessageBody']['SamplingInformation'], tagPath, url, timestamp, utoken, stoken)

        for iuCount in range(len(msg['MessageBody']['InspectedUnits'])):
            for key in msg['MessageBody']['InspectedUnits'][iuCount]:
                tagPath_UPN = tagPath + 'Unit_Position_' + str(msg['MessageBody']['InspectedUnits'][iuCount]['UnitPositionNumber']) + '.'
                if key != 'Inspections' and key != 'UnitPositionNumber':
                    tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount], tagPath_UPN, url, timestamp, utoken, stoken)
                elif key == 'Inspections':
                    for iCount in range(len(msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'])):
                        for key in msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]:
                            if key != 'Measurements' and key != 'DefectsFound' and key != 'Symptoms':
                                tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount], tagPath_UPN, url, timestamp, utoken, stoken)
                            elif key == 'Measurements':
                                for mCount in range(len(msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Measurements'])):
                                    for key in msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Measurements'][mCount]:
                                        tagPath_MN = tagPath_UPN + str(msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Measurements'][mCount]['MeasurementName'])+'.'
                                        if key != 'MeasurementName' and key != 'CRDs':
                                            tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Measurements'][mCount], tagPath_MN, url, timestamp, utoken, stoken)
                            elif key == 'DefectsFound':
                                for dCount in range(len(msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['DefectsFound'])):
                                    for key in msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['DefectsFound'][dCount]:
                                        if key != 'ComponentOfInterest':
                                            tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['DefectsFound'][dCount], tagPath_UPN, url, timestamp, utoken, stoken)
                                        elif key == 'ComponentOfInterest':
                                            for key in msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['DefectsFound'][dCount]['ComponentOfInterest']:
                                                tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['DefectsFound'][dCount]['ComponentOfInterest'], tagPath_UPN, url, timestamp, utoken, stoken)

                            elif key == 'Symptoms':
                                for sCount in range(len(msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Symptoms'])):
                                    for key in msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Symptoms'][sCount]:
                                        tagValue(key, msg['MessageBody']['InspectedUnits'][iuCount]['Inspections'][iCount]['Symptoms'][sCount], tagPath_UPN, url, timestamp, utoken, stoken)
    except Exception as e:
        print(e)
    revokeSessionToken(utoken,stoken)
        
################## URL AND FUNCTION ##################

url = 'https://historian.domain.com:<port>/api/v1/'
storeDataOlympus(url)